In [49]:
import pandas as pd
import numpy as np
import nltk
import re
import string
import utilities as ut



In [50]:
# Ejemplo de recomendación de usuarios

database=pd.read_csv("database_corregido.csv")

def RecomendacionUsuario(Top, ID_usuario):
    ID_usuario = ID_usuario - 1
    print('Usuario:', ut.carga_datos(database).columns[ID_usuario])
    RecomendacionUsuario = ut.carga_datos(database).iloc[(-ut.carga_datos(database).iloc[:, ID_usuario]).argsort()[1:(Top+1)].values, ID_usuario]
    print ('\n Los usuarios mas compatibles son:')
    return RecomendacionUsuario



In [52]:
RecomendacionUsuario(10, 1)

AttributeError: module 'utilities' has no attribute 'carga_datos'